In [2]:
# fix imports
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
import torch
from notebooks.experiment_robust import load_robust_experiment
from notebooks.experiment_torch import load_torchvision_experiment

model, dl_train, dl_eval = load_robust_experiment("Standard_R50", "imagenet")
# model, dl_train, dl_eval = load_torchvision_experiment("alexnet", batch_size=32)

/home/fre.gilad/miniforge3/envs/adv_hw/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/fre.gilad/miniforge3/envs/adv_hw/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Creating Dataset:   0%|          | 0/10000 [00:00<?, ?it/s]

Creating Dataset:   0%|          | 0/5000 [00:00<?, ?it/s]

Extracting Samples:   0%|          | 0/10 [00:00<?, ?it/s]

Extracting Samples:   0%|          | 0/5 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from ulib.attack import StopCriteria
from ulib.attacks.usgd import USGD
from ulib.pert_module import PertModule
from ulib import utils

stop = StopCriteria(max_epochs=10, max_time=600)
pert_model = PertModule(model, data_shape=dl_train.data.shape[1:], eps=8 / 255, random_init=True)
optimizer = torch.optim.Adam(pert_model.parameters(), lr=1e-2, maximize=True)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=100, T_mult=1)
grad_scaler = torch.GradScaler(device=pert_model.device.type)

attack = USGD(
    pert_model=pert_model,
    optimizer=optimizer,
    criterion=criterion,
    # scheduler=scheduler,
    grad_scaler=grad_scaler,
    skip_already_fooled=False,
    sched_on_batch=True,
)

pert = attack.fit(dl_train, dl_eval, stop)

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
attack.close()

In [ ]:
from ulib import eval

eval.full_analysis(pert_model, dl_eval)